In [1]:
import pandas as pd
from happiness_mining import scoring as hs
from happiness_mining import nlp

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jonghopark/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = "../data/raw/mastodon.csv"
df = pd.read_csv(path)
df = df.loc[df["language"]=="en"]
df['token'] = df['content'].apply(nlp.create_tokens)

/Users/jonghopark/Documents/GitHub/social-media-analytics-on-cloud/script/happiness_mining/nlp.py:33: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tmp = BeautifulSoup(text, features="html.parser").text


In [3]:
model = hs.happiness_score()

In [4]:
import pyarrow as pa

In [5]:
words = []
df["t_score"] = 0
df["w_score"] = 0

for i, row in df.iterrows():
    words += row["token"]
    score = model.scoring_by_text(row["token"])
    df.loc[i, "t_score"] = score

model.set_whole_words(words)
for i, row in df.iterrows():
    score = model.scoring_in_whole(row["token"])
    df.loc[i, "w_score"] = score

In [6]:
import csv

In [7]:
outpath = "../data/curated/mastodon_processed.csv"
df[["created_at","hashtags","language","token","t_score","w_score"]].to_csv(outpath)

In [14]:
df = pd.read_csv(outpath, index_col=0) 
df["hashtags"] = df["hashtags"].str.strip('][').str.split(', ')
df["token"] = df["token"].str.strip('][').str.split(', ')

In [15]:
df

,created_at,hashtags,language,token,t_score,w_score
0,2023-04-26 04:27:53+00:00,['nba'],en,"['team', 'could', 'trick', 'cp3', 'sun', 'nba']",6.526667,5.904248
1,2023-04-26 04:27:51+00:00,[],en,"['piec', 'damag', 'storag', 'scratch', 'much',...",5.708889,6.130553
2,2023-04-26 04:27:54+00:00,"['wanteddeadoralive', '9yearsofwarinukraine', ...",en,"['🇺🇸🇺🇦draft', 'resolut', '‘', 'ukrainian', 'vi...",6.040000,6.584891
3,2023-04-26 04:27:37+00:00,['news'],en,"['afl', 'regret', 'past', 'concuss', 'manag', ...",5.530000,5.694183
4,2023-04-26 04:27:55+00:00,[],en,"['whole', 'spiritu', 'transform', 'bring', 'us...",6.070000,6.201230
...,...,...,...,...,...,...
87271,2023-04-28 09:15:20+00:00,['auspol'],en,"['auspol', 'pm', ',', 'nightclub', 'mogul', 'f...",6.284000,6.742366
87273,2023-04-28 09:16:03+00:00,[],en,"['franc', ',', 'citi', 'smaller', 'kirikiriroa...",4.900000,5.395647
87275,2023-04-28 09:16:02+00:00,"['clojure', 'databases']",en,"['xtdb', 'open-sourc', 'datom', 'altern', 'clo...",0.000000,0.000000
87277,2023-04-28 09:15:22+00:00,"['database', 'analytics', 'datamanagement', 's...",en,"['make', 'apach', 'kafka', 'realli', 'understa...",6.145000,6.002455
